# This is the Baseline Model for doing Machine Learning in Birdcall Recognition


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import plotly.express as px
import librosa

import pywt
from statsmodels.robust import mad
from warnings import filterwarnings; filterwarnings('ignore')

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from scipy.io import wavfile
import subprocess

from tqdm.notebook import tqdm
import librosa

PATH = '../input/birdsong-recognition/'
TEST_FOLDER = '../input/birdsong-recognition/test_audio/'
os.listdir(PATH)
RANDOM_SEED = 4444

In [ ]:
train = pd.read_csv(os.path.join(PATH, 'train.csv'))
test = pd.read_csv(os.path.join(PATH, 'test.csv'))
submission = pd.read_csv(os.path.join(PATH, 'sample_submission.csv'))
example_test_audio_summary = pd.read_csv(os.path.join(PATH, 'example_test_audio_summary.csv'))
example_test_audio_metadata = pd.read_csv(os.path.join(PATH, 'example_test_audio_metadata.csv'))

In [9]:
def load_test_clip(path, start_time, duration=5):
    return librosa.load(path, offset=start_time, duration=duration)[0]

def make_prediction(y, le, model):
    feats = np.array([np.min(y), np.max(y), np.mean(y), np.std(y)]).reshape(1, -1)
    return le.inverse_transform(model.predict(feats))[0]

## Below is for the train attribute generation. Note these values selected are really **naive**.
*TODO: OPTIMIZE IT*

In [ ]:
train = pd.DataFrame(columns = ['min', 'max', 'mean', 'std', 'target'])
nRows = 0
for bird in tqdm(os.listdir(os.path.join(PATH, 'train_audio'))):
    for audio in  os.listdir(os.path.join(PATH, 'train_audio', bird)):
        path = os.path.join(PATH, 'train_audio', bird, audio)
        subprocess.call(['ffmpeg', '-y', '-i', f'{path}', f'../temp/temp.wav'])
        _, y = wavfile.read('../temp/temp.wav')
        train.loc[nRows] = [np.min(y), np.max(y), np.mean(y), np.std(y), bird]
        nRows += 1

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['target'] = le.fit_transform(train['target'].values)

# The Model that we used here is the RandomForest Classifier 
*TODO: OPTIMIZE IT*

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth = 6, random_state = RANDOM_SEED)
model.fit(train.drop(columns = 'target'), train['target'].values) 

In [ ]:
Note Below, it trys to read the test.csv's full data. If the test.csv is not complete, 

In [ ]:
try:
    preds = []
    for index, row in test.iterrows():
        # Get test row information
        site = row['site']
        start_time = row['seconds'] - 5
        row_id = row['row_id']
        audio_id = row['audio_id']

        # Get the test sound clip
        if site == 'site_1' or site == 'site_2':
            y = load_test_clip(TEST_FOLDER + audio_id + '.mp3', start_time)
        else:
            y = load_test_clip(TEST_FOLDER + audio_id + '.mp3', 0, duration=None)

        # Make the prediction
        pred = make_prediction(y, le, model)

        # Store prediction
        preds.append([row_id, pred])
    
except:
     preds = pd.read_csv('../input/birdsong-recognition/sample_submission.csv')
print(preds)
preds = pd.DataFrame(preds, columns=['row_id', 'birds'])

In [ ]:
preds.to_csv('submission.csv', index=False) 